In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from natsort import natsorted
import numpy as np
import matplotlib.animation as animation
import xarray as xr
import h5py
import imageio
import matplotlib
import gc
import sys
import io
import matplotlib.colors as mcolors
import matplotlib.patches as patches
from scipy.optimize import curve_fit
import scipy.integrate


# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(rf'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM
plt.rcParams.update({'font.size': 22})  

tidy3dAPI = os.environ["API_TIDY3D_KEY"]
plt.rc('font', family='Arial')


In [ ]:
# file = rf"H:\phd stuff\tidy3d\output\LSU Data\transmission_10.89.h5"
# file_host = rf"H:\phd stuff\tidy3d\output\LSU Data\transmission_10.89_host.h5"
file = rf"H:\phd stuff\tidy3d\output\LSU Data\transmission_10.89_gaussian_source_host_air.h5"
file_host = rf"H:\phd stuff\tidy3d\output\LSU Data\transmission_10.89_gaussian_source_host_decaline.h5"
with h5py.File(file, 'r') as h5f:
    data = {
            'transmission_right':h5f["transmission_right"][:],#shape (n_index, sizes, freqs)
            'transmission_left':h5f["transmission_left"][:],
            'power_far_field':h5f["power_far_field"][:],#shape (n_index, sizes, theta,phi,freqs)
            'power_far_field_ref':h5f["power_far_field_ref"][:],
            'nu(a/lambda)':h5f["nu(a/lambda)"][:],
            "raw_freqs(Hz)":h5f["raw_freqs(Hz)"][:],
            "sizes":h5f["sizes"][:],
            "cell_size":h5f["cell_size"],
            "a":h5f["a"],
            "eps":h5f["eps"][:]
                        
            }
    
with h5py.File(file_host, 'r') as h5f:
    data_host = {
            'transmission_right':h5f["transmission_right"][:],
            'transmission_left':h5f["transmission_left"][:],
            'power_far_field':h5f["power_far_field"][:],
            'power_far_field_ref':h5f["power_far_field_ref"][:],
            'nu(a/lambda)':h5f["nu(a/lambda)"][:],
            "raw_freqs(Hz)":h5f["raw_freqs(Hz)"][:],
            "sizes":h5f["sizes"][:],
            "cell_size":h5f["cell_size"],
            "a":h5f["a"],
            "eps":h5f["eps"][:],
            "eps_host":np.array([2.030625])
                        
            }

In [ ]:
np.shape(data["power_far_field_ref"])

In [ ]:
scaled_lambdas = 3.4/data['nu(a/lambda)'] * 0.8
sizes = data["sizes"]
a=3.4

In [ ]:
# #scale lambda to 80%
# dir_plot = Path(rf"H:\phd stuff\tidy3d\output\LSU Data\Host_Analsysis\20250305_n_rod_3.3_n_h_1.424\Near Field")
# dir_plot.mkdir(parents=True,exist_ok=True)

# for i,item in enumerate(sizes):
#     plt.figure(figsize=(20, 10))
#     title = rf"L_{(item*14.3/3.4):.4f}a"
#     plt.plot(scaled_lambdas,data['transmission_right'][0,i,:],label="Air")
#     plt.plot(scaled_lambdas,data_host['transmission_right'][0,i,:],label="Decaline")
#     plt.xlabel(rf'$\lambda$')
#     plt.ylabel('Transmission')
#     plt.legend()
#     plt.grid(True, which="both")
#     plt.yscale("log")
#     plt.title(title)
#     plt.savefig(rf"{dir_plot}\{title}.pdf")
#     plt.close()


In [ ]:
r_proj =  1e6
theta_proj = np.linspace(0, np.pi/2, 280)
phi_proj = np.linspace(0,2*np.pi, 280)
indices_0_30 = np.where(np.bool_(theta_proj*180/np.pi >=0) & np.bool_(theta_proj*180/np.pi <=30))[0]
indices_50_60 = np.where(np.bool_(theta_proj*180/np.pi >=50) & np.bool_(theta_proj*180/np.pi <=65))[0]

In [ ]:
r,theta,phi = r_proj,theta_proj[indices_0_30],phi_proj
# Calculate the differential surface area element in spherical coordinates
dA_1= (r ** 2) * np.sin(theta_proj) * np.diff(theta_proj)[0] * np.diff(phi)[0]
dA_2= (r ** 2) * np.sin(theta) * np.diff(theta)[0] * np.diff(phi)[0]
dA_3= (r ** 2) * np.sin(theta_proj[indices_50_60]) * np.diff(theta_proj[indices_50_60])[0] * np.diff(phi)[0]

In [ ]:
#References 
transmission_0_n_h_1 = ((data["power_far_field_ref"][indices_0_30,:,:])* dA_2[:,np.newaxis,np.newaxis]).sum(axis=(0,1))
transmission_0_n_h_1_425 = ((data_host["power_far_field_ref"][indices_0_30,:,:])* dA_2[:,np.newaxis,np.newaxis]).sum(axis=(0,1))


In [ ]:
# #Both
# dir_plot = Path(rf"H:\phd stuff\tidy3d\output\LSU Data\Host_Analsysis\20250305_n_rod_3.3_n_h_1.424\Far Field")
# dir_plot.mkdir(parents=True,exist_ok=True)
# for i,item in enumerate(sizes):
#     plt.figure(figsize=(20, 10))

#     #Decaline
#     low_lambda_indices = np.where(scaled_lambdas<=6.45)[0]
#     large_lambda_indices = np.where(scaled_lambdas>6.45)[0]
#     power_data = data_host["power_far_field"][0,i]
#     transmission_low_lambda_0_15 = (power_data[np.ix_(indices_0_30,np.arange(power_data.shape[1]),low_lambda_indices)]* dA_2[:,np.newaxis,np.newaxis]).sum(axis=(0,1))
#     transmission_low_lambda_50_60 = (power_data[np.ix_(indices_50_60,np.arange(power_data.shape[1]),low_lambda_indices)]* dA_3[:,np.newaxis,np.newaxis]).sum(axis=(0,1))
#     transmission_low_lambda = transmission_low_lambda_0_15#+transmission_low_lambda_50_60
#     transmission_high_lambda = (power_data[:,:,large_lambda_indices]* dA_1[:,np.newaxis,np.newaxis]).sum(axis=(0,1))
#     transmission = np.concatenate((transmission_high_lambda,transmission_low_lambda),axis=0)
#     # transmission_normalized_decaline = transmission/transmission_0_n_h_1_425
#     transmission_normalized_decaline =  ((power_data[indices_0_30,:,:])* dA_2[:,np.newaxis,np.newaxis]).sum(axis=(0,1))/transmission_0_n_h_1_425

#     #Air
#     low_lambda_indices = np.where(scaled_lambdas<=5.8)[0]
#     large_lambda_indices = np.where(scaled_lambdas>5.8)[0]
#     power_data = data["power_far_field"][0,i]
#     transmission_low_lambda_0_15 = (power_data[np.ix_(indices_0_30,np.arange(power_data.shape[1]),low_lambda_indices)]* dA_2[:,np.newaxis,np.newaxis]).sum(axis=(0,1))
#     transmission_low_lambda_50_60 = (power_data[np.ix_(indices_50_60,np.arange(power_data.shape[1]),low_lambda_indices)]* dA_3[:,np.newaxis,np.newaxis]).sum(axis=(0,1))
#     transmission_low_lambda = transmission_low_lambda_0_15#+transmission_low_lambda_50_60
#     transmission_high_lambda = (power_data[:,:,large_lambda_indices]* dA_1[:,np.newaxis,np.newaxis]).sum(axis=(0,1))
#     transmission = np.concatenate((transmission_high_lambda,transmission_low_lambda),axis=0)
#     # transmission_normalized = transmission/transmission_0_n_h_1 
#     transmission_normalized = ((power_data[indices_0_30,:,:])* dA_2[:,np.newaxis,np.newaxis]).sum(axis=(0,1))/transmission_0_n_h_1 


#     title = rf"L_{(item*14.3/3.4):.4f}a"
#     plt.plot(scaled_lambdas,transmission_normalized_decaline,"o",label=rf"$n_h=1.425$")
#     plt.plot(scaled_lambdas,transmission_normalized,"o",label=rf"$n_h=1$")
#     plt.xlabel(rf'$\lambda$')
#     plt.ylabel('Transmission')
#     plt.legend()
#     plt.yscale("log")
#     plt.grid(True, which="both")
#     plt.title(title)

#     plt.savefig(rf"{dir_plot}\{title}.png")
#     plt.close()


In [ ]:
#Convulution 
import scipy.ndimage
plt.rcParams.update({'font.size': 15})  
fig, ax = plt.subplots(nrows=4, ncols=3, figsize=(25, 15))
k,l=0,0
transmissionValues_air = []
transmissionValues_Decaline = []
transmissionValues_air_gaussian = []
transmissionValues_decaline_gaussian = []
for i,item in enumerate(sizes):
    if np.mod(i,3)==0 and i>0:
        k+=1
        l=0
    
    #Decaline
    power_data = data_host["power_far_field"][0,i]
    transmission_normalized_decaline =  ((power_data[indices_0_30,:,:])* dA_2[:,np.newaxis,np.newaxis]).sum(axis=(0,1))/transmission_0_n_h_1_425
    transmissionValues_Decaline.append(transmission_normalized_decaline)
    #Air
    power_data = data["power_far_field"][0,i]
    transmission_normalized = ((power_data[indices_0_30,:,:])* dA_2[:,np.newaxis,np.newaxis]).sum(axis=(0,1))/transmission_0_n_h_1 
    transmissionValues_air.append(transmission_normalized)


    # Apply Gaussian smoothing
    sigma = 15  # Adjust for more/less smoothing
    smoothed_transmission = scipy.ndimage.gaussian_filter1d(transmission_normalized, sigma)
    transmissionValues_air_gaussian.append(smoothed_transmission)
    
    smoothed_transmission_host = scipy.ndimage.gaussian_filter1d(transmission_normalized_decaline, 5)
    transmissionValues_decaline_gaussian.append(smoothed_transmission_host)

    title = rf"L_{(item*14.3/3.4):.4f}a"
    ax[k,l].plot(scaled_lambdas,transmission_normalized,"o",label=rf"$n_h=1$")
    ax[k,l].plot(scaled_lambdas,smoothed_transmission)
    ax[k,l].plot(scaled_lambdas,transmission_normalized_decaline,"o",label=rf"$n_h=1.425$")
    ax[k,l].plot(scaled_lambdas,smoothed_transmission_host)
    ax[k,l].set_yscale("log")
    ax[k,l].grid(True, which="both")
    ax[k,l].set_title(title)
    ax[k,l].legend()
    l+=1

fig.supylabel("Transmission", fontsize=14)
fig.supxlabel(rf"$\lambda$", fontsize=14)
plt.tight_layout()
plt.show()
# plt.close()






In [ ]:
lambdas_gap = np.where(np.bool_(scaled_lambdas>=5) & np.bool_(scaled_lambdas<7))[0]

In [ ]:
min_transmission = np.min(np.array(transmissionValues_air)[:,list(lambdas_gap)], axis=1)

In [ ]:
def fit_line(x,m):
    return m*x 
initial_guess = [-1]

plt.plot( sizes[1:]*14.3*0.8,min_transmission[1:],"o")
popt, _ = curve_fit(fit_line, sizes[1:]*14.3*0.8, np.log(min_transmission[1:]), p0=initial_guess)
plt.plot( sizes[1:]*14.3*0.8,np.exp(fit_line(sizes[1:]*14.3*0.8,popt[0])), label=rf"$L_B$={-1/popt[0]:.4f}")
plt.xlabel("L")
plt.ylabel("Transmission")
plt.yscale("log")
plt.legend()
plt.show()


In [ ]:
size_str = ''
for item in sizes:
     size_str+= rf"L_{item*0.8*14.3:.4f} "


In [ ]:
arrays = ["transmissionValues_air","transmissionValues_Decaline","transmissionValues_air_gaussian","transmissionValues_decaline_gaussian"]

for element in arrays: 
    data_new = np.transpose(eval(element))
    np.savetxt(rf"H:\phd stuff\tidy3d\output\LSU Data\txtData\{element}.txt", data_new, fmt="%.18f", delimiter=" ", header=size_str, comments="")


In [20]:
transmission = np.transpose(data_host['transmission_right'].squeeze())

np.savetxt(rf"H:\phd stuff\tidy3d\output\LSU Data\txtData\transmission_n_h_1.425.txt", transmission , fmt="%.18f", delimiter=" ", header=size_str, comments="")
